In [ ]:
# This cell makes sure modules are auto-loaded when you change external python files
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import torch
from torch import optim

from transformers import (
    TrainingArguments, 
    Trainer, 
)

In [ ]:
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

In [ ]:
if torch.cuda.is_available():
  print("Good to go!")
else:
  print("Please connect to a GPU to run this notebook.")

In [ ]:
from sa import SATask
from nli import MNLITask
from ner import NERTask
from qa import SQuADTask

def retrieve_model(fine_tunning_task, model_name="roberta-base"):
    if fine_tunning_task == "sa":
        task = SATask(model_name)
    elif fine_tunning_task == "nli":
        task = MNLITask(model_name)
    elif fine_tunning_task == "ner":
        task = NERTask(model_name)
    elif fine_tunning_task == "qa":
        task = SQuADTask(model_name)
    else:
        raise ValueError("Invalid task")

    return task

In [ ]:
model_name = "roberta-base"
fine_tunning_task = "ner"
task = retrieve_model(fine_tunning_task, model_name)

In [ ]:
training_args = TrainingArguments(
    output_dir="{}-finetuned-{}".format(model_name, fine_tunning_task),
    learning_rate=2e-5,
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    # weight_decay=0.01,
    evaluation_strategy = "epoch", # "no" to avoid evaluation
    save_strategy = "epoch", # "no" to avoid saving
    logging_steps=500,
    report_to="tensorboard",
    logging_dir="./tensorboard/{}-finetuned-{}".format(model_name, fine_tunning_task),
)

# Huggingface optimizers: https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/optimizer_schedules#optimization
# Huggingface Schedulers: https://huggingface.co/docs/transformers/v4.39.3/en/main_classes/optimizer_schedules#schedules
# E.g.
# from transformers import AdamW, get_linear_schedule_with_warmup
# optimizer = AdamW(model.parameters(), lr=training_args.learning_rate)
# num_warmup_steps = training_args.num_train_epochs * len(train_dataset) // training_args.per_device_train_batch_size  # Adjust warmup based on your training setup
# scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=training_args.num_train_epochs * len(train_dataset) // training_args.per_device_train_batch_size)


# PyTorch optimizers: https://pytorch.org/docs/stable/optim.html#algorithms
# PyTorch schedulers: https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
optimizer = optim.SGD(task.model.parameters(), lr=training_args.learning_rate)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)


trainer = Trainer(
    task.model,
    args=training_args,
    optimizers=(optimizer, scheduler),
    train_dataset=task.encoded_dataset["train"],
    eval_dataset=task.encoded_dataset["validation"],
    tokenizer=task.tokenizer,
    data_collator=task.data_collator,
    compute_metrics=task.compute_metrics
)

In [ ]:
trainer.train()
# trainer.save_model("models/{}-finetuned-{}".format(model_name, fine_tunning_task))

In [ ]:
# eval_result = trainer.evaluate()